In [1]:
import sys
import os
from tempfile import mkdtemp
sys.path.insert(0, '..')

import numpy as np
from process_bigraph import Composite, pp
from process_bigraph.experiments.parameter_scan import RunProcess

from biosimulator_processes import CORE 


Available processes:
[ 'console-emitter',
  'ram-emitter',
  'composite',
  'cobra-process',
  'copasi-process',
  'tellurium-process',
  'utc-amici',
  'utc-copasi',
  'utc-tellurium',
  'smoldyn-process',
  'copasi-step',
  'tellurium-step',
  'amici-step',
  'plotter',
  'plotter2d',
  'utc-comparator']


1. Define inputs

In [2]:
omex_dir = '../test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator'
omex_fp = omex_dir + '.omex'
sbml_fp = os.path.join(omex_dir, 'BIOMD0000000012_url.xml')
sedml_fp = os.path.join(omex_dir, 'simulation.sedml')
reports_path = os.path.join(omex_dir, 'reports.h5')
simulators = ['amici', 'copasi', 'tellurium']

2. Define composition document (manuscript?)

In [3]:
manuscript = {
    'amici': {
        '_type': 'step',
        'address': 'local:utc-amici',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['amici_floating_species_store'],
            'model_parameters': ['amici_model_parameters_store'],
            'reactions': ['amici_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['amici_floating_species_store']
        }
    },
    'copasi': {
        '_type': 'step',
        'address': 'local:utc-copasi',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['copasi_floating_species_store'],
            'model_parameters': ['copasi_model_parameters_store'],
            'reactions': ['copasi_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['copasi_floating_species_store']
        }
    },
    'tellurium': {
        '_type': 'step',
        'address': 'local:utc-tellurium',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['tellurium_floating_species_store'],
            'model_parameters': ['tellurium_model_parameters_store'],
            'reactions': ['tellurium_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['tellurium_floating_species_store']
        }
    },
    'comparison': {
        '_type': 'step',
        'address': 'local:utc-comparator',
        'config': {
            'simulators': ['amici', 'copasi', 'tellurium'],
        },
        'inputs': {
            'time': ['time_store'],
            'amici_floating_species': ['amici_floating_species_store'],
            'copasi_floating_species': ['copasi_floating_species_store'],
            'tellurium_floating_species': ['tellurium_floating_species_store'],
        },
        'outputs': {
            'results': ['results_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'results': 'tree[float]',
            }
        },
        'inputs': {
            'results': ['results_store'],
        }
    }
}


output_bridge = {
    'inputs': {
        'results': ['results_store']
    },
    'outputs': {
        'results': ['results_store']
    }
}

3. Create composition with doc specifications

In [4]:
composition_config = {'state': manuscript, 'bridge': output_bridge}
composition = Composite(
    config=composition_config,
    core=CORE
)

2024-06-06 03:50:46.861 - amici.sbml_import - INFO - Finished importing SBML                         (6.23E-02s)
2024-06-06 03:50:46.999 - amici.sbml_import - INFO - Finished processing SBML observables            (1.35E-01s)
2024-06-06 03:50:47.003 - amici.sbml_import - INFO - Finished processing SBML event observables      (1.43E-06s)
2024-06-06 03:50:47.051 - amici.de_model - INFO - Finished computing xdot                            (3.89E-03s)
2024-06-06 03:50:47.056 - amici.de_model - INFO - Finished computing x0                              (2.27E-03s)
2024-06-06 03:50:47.099 - amici.de_model - INFO - Finished computing w                               (4.05E-02s)
2024-06-06 03:50:48.253 - amici.de_export - INFO - Finished generating cpp code                      (1.15E+00s)
2024-06-06 03:51:02.309 - amici.de_export - INFO - Finished compiling cpp code                       (1.41E+01s)


In [5]:
dir(composition)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply_updates',
 'bridge',
 'bridge_updates',
 'composition',
 'config',
 'config_schema',
 'core',
 'cycle_step_state',
 'determine_steps',
 'edge_paths',
 'emitter_paths',
 'expire_process_paths',
 'find_instance_paths',
 'front',
 'gather_results',
 'global_time_precision',
 'initial_state',
 'inputs',
 'interface',
 'invoke',
 'merge',
 'node_dependencies',
 'outputs',
 'process_paths',
 'process_schema',
 'process_update',
 'reset_step_state',
 'run',
 'run_process',
 'run_steps',
 'state',
 'step_dependencies',
 'step_paths',
 'step_triggers',
 'steps_remaining',
 'steps_run',
 'to_run',
 'trigger_state',
 'trigger

In [6]:
pp(composition.state)

{ 'amici': { '_inputs': { 'floating_species': 'tree[float]',
                          'model_parameters': { 'KM': { '_apply': 'set',
                                                        '_type': 'float'},
                                                'a0_tr': { '_apply': 'set',
                                                           '_type': 'float'},
                                                'a_tr': { '_apply': 'set',
                                                          '_type': 'float'},
                                                'alpha': { '_apply': 'set',
                                                           '_type': 'float'},
                                                'alpha0': { '_apply': 'set',
                                                            '_type': 'float'},
                                                'average mRNA life time': { '_apply': 'set',
                                                                            '_type

In [7]:
for simulator in simulators:
    print(f'{simulator}:')
    pp(composition.state[simulator]['instance'].initial_state())
    print('\n')


amici:
{ 'floating_species': { 'PX': 0.0,
                        'PY': 0.0,
                        'PZ': 0.0,
                        'X': 0.0,
                        'Y': 20.0,
                        'Z': 0.0},
  'model_parameters': { 'KM': 40.0,
                        'a0_tr': 0.0,
                        'a_tr': 0.0,
                        'alpha': 216.404,
                        'alpha0': 0.2164,
                        'average mRNA life time': 0.0,
                        'beta': 0.2,
                        'k_tl': 0.0,
                        'kd_mRNA': 0.0,
                        'kd_prot': 0.0,
                        'mRNA half life': 2.0,
                        'n': 2.0,
                        'protein half life': 10.0,
                        'tps_active': 0.5,
                        'tps_repr': 0.0005,
                        'translation efficiency': 20.0},
  'reactions': { 'degradation of CI': 0.0,
                 'degradation of CI transcripts': 0.0,
      

4. Run composition

In [8]:
composition.bridge

{'inputs': {'results': ['results_store']},
 'outputs': {'results': ['results_store']}}

In [9]:
composition.run(1)

In [10]:
composition_results = composition.gather_results()

In [11]:
composition_results

{('emitter',): []}

In [12]:
composition.state

{'amici': {'_type': 'step',
  'address': 'local:utc-amici',
  'config': {'model': {'model_source': '/tmp/tmph5kky_k7/BIOMD0000000012_url.xml',
    'model_id': '',
    'model_language': 'sbml',
    'model_name': 'composite_process_model',
    'model_changes': {},
    'model_units': {}},
   'time_config': {'duration': None,
    'num_steps': None,
    'step_size': None,
    'output_start_time': None,
    'initial_time': None},
   'species_context': 'concentrations',
   'working_dir': '',
   'method': 'deterministic',
   'model_output_dir': '/tmp/tmpsm6ikrgc',
   'observables': None,
   'constant_parameters': None,
   'sigmas': None},
  'inputs': {'time': ['time_store'],
   'floating_species': ['amici_floating_species_store'],
   'model_parameters': ['amici_model_parameters_store'],
   'reactions': ['amici_reactions_store']},
  'outputs': {'time': ['time_store'],
   'floating_species': ['amici_floating_species_store']},
  'instance': <biosimulator_processes.processes.amici_process.UtcAmici

In [13]:
from biosimulator_processes.execute import exec_utc_comparison

In [14]:
utc_comparison_result = exec_utc_comparison(omex_fp, simulators)
_c = utc_comparison_result.copy()
for spec_name, result in utc_comparison_result['results'].items():
    _c['results'][spec_name].pop('output_data')
pp(_c)

2024-06-06 03:51:02.694 - amici.sbml_import - INFO - Finished loading SBML                           (7.74E-03s)
2024-06-06 03:51:02.744 - amici.sbml_import - INFO - Finished importing SBML                         (4.82E-02s)
2024-06-06 03:51:02.782 - amici.sbml_import - INFO - Finished processing SBML observables            (3.52E-02s)
2024-06-06 03:51:02.786 - amici.sbml_import - INFO - Finished processing SBML event observables      (1.86E-06s)
2024-06-06 03:51:02.826 - amici.de_model - INFO - Finished computing xdot                            (3.15E-03s)
2024-06-06 03:51:02.831 - amici.de_model - INFO - Finished computing x0                              (2.40E-03s)
2024-06-06 03:51:02.849 - amici.de_model - INFO - Finished computing w                               (1.53E-02s)
2024-06-06 03:51:03.640 - amici.de_export - INFO - Finished generating cpp code                      (7.86E-01s)
2024-06-06 03:51:15.918 - amici.de_export - INFO - Finished compiling cpp code                  

{ 'id': 'repressilator-comparison',
  'results': { 'LacI mRNA': { 'mse': { 'amici': { 'amici': 0.0,
                                                  'copasi': 1.0885014890955766,
                                                  'tellurium': 307.2029720928004},
                                       'copasi': { 'amici': 1.0885014890955766,
                                                   'copasi': 0.0,
                                                   'tellurium': 305.76255388233653},
                                       'tellurium': { 'amici': 307.2029720928004,
                                                      'copasi': 305.76255388233653,
                                                      'tellurium': 0.0}},
                              'prox': { 'amici': { 'amici': True,
                                                   'copasi': False,
                                                   'tellurium': False},
                                        'copasi': { 'amici':

In [15]:
utc_comparison_result['id']

'repressilator-comparison'

In [16]:
from biosimulator_processes.api.instance import UtcComparisonDocument
doc = UtcComparisonDocument(model_source=omex_fp)

In [17]:
doc

{'state': {'amici': {'_type': 'step',
   'address': 'local:utc-amici',
   'config': {'model': {'model_source': '../test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator.omex'}},
   'inputs': {'time': ['time_store'],
    'floating_species': ['amici_floating_species_store'],
    'model_parameters': ['amici_model_parameters_store'],
    'reactions': ['amici_reactions_store']},
   'outputs': {'time': ['time_store'],
    'floating_species': ['amici_floating_species_store']}},
  'copasi': {'_type': 'step',
   'address': 'local:utc-copasi',
   'config': {'model': {'model_source': '../test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator.omex'}},
   'inputs': {'time': ['time_store'],
    'floating_species': ['copasi_floating_species_store'],
    'model_parameters': ['copasi_model_parameters_store'],
    'reactions': ['copasi_reactions_store']},
   'outputs': {'time': ['time_store'],
    'floating_species': ['copasi_floating_species_store']}},
  'tellurium': {'_type': 'step'

Perform an entire sbml source directory scan with the comparison:

In [18]:
from biosimulator_processes.execute import sbml_utc_comparison_scan

source_dir = '../test_suite/examples/sbml-core'
scan_results = sbml_utc_comparison_scan(source_dir, simulators)

# successfully run
scan_results.keys()

2024-06-06 03:51:16.104 - amici.sbml_import - INFO - Finished loading SBML                           (6.79E-03s)
2024-06-06 03:51:16.155 - amici.sbml_import - INFO - Finished importing SBML                         (4.86E-02s)
2024-06-06 03:51:16.188 - amici.sbml_import - INFO - Finished processing SBML observables            (3.02E-02s)
2024-06-06 03:51:16.191 - amici.sbml_import - INFO - Finished processing SBML event observables      (1.40E-06s)
2024-06-06 03:51:16.232 - amici.de_model - INFO - Finished computing xdot                            (3.54E-03s)
2024-06-06 03:51:16.239 - amici.de_model - INFO - Finished computing x0                              (2.29E-03s)
2024-06-06 03:51:16.255 - amici.de_model - INFO - Finished computing w                               (1.46E-02s)
2024-06-06 03:51:17.043 - amici.de_export - INFO - Finished generating cpp code                      (7.83E-01s)
2024-06-06 03:51:29.305 - amici.de_export - INFO - Finished compiling cpp code                  

dict_keys(['Elowitz-Nature-2000-Repressilator', 'Vilar-PNAS-2002-minimal-circardian-clock.omex', 'Edelstein-Biol-Cybern-1996-Nicotinic-excitation.omex', 'Vilar-PNAS-2002-minimal-circardian-clock-discrete-SSA', 'Tomida-EMBO-J-2003-NFAT-translocation.omex', 'Elowitz-Nature-2000-Repressilator.omex', 'Parmar-BMC-Syst-Biol-2017-iron-distribution.omex', 'Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.omex', 'Caravagna-J-Theor-Biol-2010-tumor-suppressive-oscillations', 'Szymanska-J-Theor-Biol-2009-HSP-synthesis', 'Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-Fehlberg.omex', 'Tomida-EMBO-J-2003-NFAT-translocation', 'Varusai-Sci-Rep-2018-mTOR-signaling-LSODA-LSODAR-SBML', 'Varusai-Sci-Rep-2018-mTOR-signaling-LSODA-LSODAR-SBML.omex', 'Caravagna-J-Theor-Biol-2010-tumor-suppressive-oscillations.omex', 'Szymanska-J-Theor-Biol-2009-HSP-synthesis.omex', 'Parmar-BMC-Syst-Biol-2017-iron-distribution', 'Vilar-PNAS-2002-minimal-circardian-clock-continuous.omex', 'Vilar-PNAS-2002-mi

In [19]:
scan_results['Parmar-BMC-Syst-Biol-2017-iron-distribution.omex']

{'error': 'At least one simulator did not support Parmar-BMC-Syst-Biol-2017-iron-distribution.omex'}

In [20]:
scan_results['Szymanska-J-Theor-Biol-2009-HSP-synthesis.omex']

{'results': {'Hsp70': {'mse': {'copasi': {'copasi': 0.0,
     'amici': 1.8981187389502497e+43,
     'tellurium': 17.60959438922604},
    'amici': {'copasi': 1.8981187389502497e+43,
     'amici': 0.0,
     'tellurium': 1.8981187389502497e+43},
    'tellurium': {'copasi': 17.60959438922604,
     'amici': 1.8981187389502497e+43,
     'tellurium': 0.0}},
   'prox': {'copasi': {'copasi': True, 'amici': False, 'tellurium': False},
    'amici': {'copasi': False, 'amici': True, 'tellurium': False},
    'tellurium': {'copasi': False, 'amici': False, 'tellurium': True}},
   'output_data': {'amici_floating_species': {'Hsp70': array([2.73546   , 2.73227442, 2.72398478, ..., 4.75360269, 4.75325444,
            4.75290689]),
     'HSF': array([0.332019  , 0.33596875, 0.3437789 , ..., 0.45866989, 0.45871426,
            0.45875855]),
     'S': array([3.09000000e-17, 1.75938445e-02, 2.61231852e-02, ...,
            3.27473126e-01, 3.27485913e-01, 3.27498676e-01]),
     'Hsp70_HSF': array([76.4593    ,

In [21]:
for f in scan_results.keys():
    if not 'error' in scan_results[f].keys():
        print(f)

Elowitz-Nature-2000-Repressilator
Elowitz-Nature-2000-Repressilator.omex
Caravagna-J-Theor-Biol-2010-tumor-suppressive-oscillations
Szymanska-J-Theor-Biol-2009-HSP-synthesis
Varusai-Sci-Rep-2018-mTOR-signaling-LSODA-LSODAR-SBML
Varusai-Sci-Rep-2018-mTOR-signaling-LSODA-LSODAR-SBML.omex
Caravagna-J-Theor-Biol-2010-tumor-suppressive-oscillations.omex
Szymanska-J-Theor-Biol-2009-HSP-synthesis.omex


In [22]:
from biosimulator_processes.execute import exec_utc_comparison

varusai_archive = os.path.join(source_dir, 'Varusai-Sci-Rep-2018-mTOR-signaling-LSODA-LSODAR-SBML.omex')
caravagna_utc_comparison = exec_utc_comparison(varusai_archive, simulators)

2024-06-06 03:53:09.031 - amici.sbml_import - INFO - Finished loading SBML                           (1.40E-02s)
2024-06-06 03:53:09.095 - amici.sbml_import - INFO - Finished importing SBML                         (6.11E-02s)
2024-06-06 03:53:09.195 - amici.sbml_import - INFO - Finished processing SBML observables            (9.73E-02s)
2024-06-06 03:53:09.198 - amici.sbml_import - INFO - Finished processing SBML event observables      (1.61E-06s)
2024-06-06 03:53:09.277 - amici.de_model - INFO - Finished computing xdot                            (5.83E-03s)
2024-06-06 03:53:09.282 - amici.de_model - INFO - Finished computing x0                              (2.29E-03s)
2024-06-06 03:53:09.328 - amici.de_model - INFO - Finished computing w                               (4.35E-02s)
2024-06-06 03:53:10.441 - amici.de_export - INFO - Finished generating cpp code                      (1.11E+00s)
2024-06-06 03:53:23.530 - amici.de_export - INFO - Finished compiling cpp code                  